# Semantic Heterogeneous Database Simulations

Let's start generating random records and semantic operations, which will be used to execute performance tests

In [1]:
import time
import json
import random
import math
import pandas as pd
import numpy as np
from pymongo import MongoClient
from database_generator import DatabaseGenerator
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
print(pd.__version__)

1.3.1


## Load Phase

Inserting all records generated in the semantic heterogeneous database. Please note PyMongo library may diminish performance of insertions. However, because the simulator internally uses it, it is fair to also use it on our baseline test, so these delays might net. 

Let's decide all variables for this test

In [3]:
number_of_records = 100000
number_of_versions = 5
number_of_fields = 11
number_of_values_in_domain=20

number_of_tests = 5
confidence_interval = 0.95

host = 'localhost'

performance_results = pd.DataFrame()

### First scenario
Inserting all records and adding the semantic operations afterwards. 

In [4]:
def first_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    d.collection.insert_many_by_dataframe(records, 'valid_from_date')

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:    
        d.collection.execute_operation(operation[0],operation[1],operation[2])
    
    end = time.time()
    d.destroy()
    return (end - start)

In [5]:
for i in range(number_of_tests):
    time_taken = first_scenario()
    performance_results = performance_results.append({'stage' : 'LoadingPhase', 'experiment': 'insert-first','time':time_taken}, ignore_index=True)

In [6]:
performance_results

,stage,experiment,time
0,LoadingPhase,insert-first,19.609344
1,LoadingPhase,insert-first,19.462202
2,LoadingPhase,insert-first,19.905794
3,LoadingPhase,insert-first,22.676517
4,LoadingPhase,insert-first,22.605921


### Second Scenario
Loading records only after inserting semantic operations in the collection

In [7]:
def second_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    d.collection.insert_many_by_dataframe(records.head(10), 'valid_from_date') #initial insert

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:          
        d.collection.execute_operation(operation[0],operation[1],operation[2])

    d.collection.insert_many_by_dataframe(records.head(-10), 'valid_from_date')
    
    end = time.time()
    d.destroy()
    return (end - start)

In [8]:
for i in range(number_of_tests):
    time_taken = second_scenario()
    performance_results = performance_results.append({'stage' : 'LoadingPhase', 'experiment': 'operations-first','time':time_taken}, ignore_index=True)

In [9]:
performance_results

,stage,experiment,time
0,LoadingPhase,insert-first,19.609344
1,LoadingPhase,insert-first,19.462202
2,LoadingPhase,insert-first,19.905794
3,LoadingPhase,insert-first,22.676517
4,LoadingPhase,insert-first,22.605921
5,LoadingPhase,operations-first,12.405283
6,LoadingPhase,operations-first,11.088462
7,LoadingPhase,operations-first,10.675926
8,LoadingPhase,operations-first,9.451641
9,LoadingPhase,operations-first,12.907995


### Baseline Scenario

In a common database scenario, the records would be just inserted as they were generated (in raw format). User would have to deal with heterogeneity afterwards, in the querying fase. Therefore, for the loading phase, only generate raw records and bulk insert into the database. 

In [10]:
def third_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    
    client = MongoClient(host)
    client[d.database_name][d.collection_name].insert_many(d.records)
    
    end = time.time()
    d.destroy()
    return (end - start)

for i in range(number_of_tests):
    time_taken = third_scenario()
    performance_results = performance_results.append({'stage' : 'LoadingPhase', 'experiment': 'baseline','time':time_taken}, ignore_index=True)

## Querying tests

For the query tests, it does not matter which database (from first loading phase or from the second) is used. Both of them posess the same number of records, fields and domain values. Let's now analyse statistics in six different scenarios, just as in YCDB benchmark.

In [11]:
def generate_test_database_preinsert():
    ### Generate database just as before
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    
    d.collection.insert_many_by_dataframe(records, 'valid_from_date')

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:    
        d.collection.execute_operation(operation[0],operation[1],operation[2])  
       
    
    return d

In order to correctly compare performance between the developed system and the baseline (an ordinary document-oriented database), this test performs a query rewriting before starting the time counter. It is important to notice, however, this test compares time performance only, regardless of usability gains achieved by this system. This query rewriting would have to be manually performed by the user, which would potentially also spend time.

In [12]:
def rewrite_query_baseline(query, database_generator):
    ors = list()

    for operation in database_generator:
        operationType, version_date, arguments = operation

        if operationType == 'grouping':
            pass
        elif operationType == 'translation':
            oldValue = arguments['from']
            newValue = arguments['to']
            field = arguments['field']

            if field not in query:
                continue
            else:
                ands = [{field : newValue}]
                

        else:
            raise BaseException('OperationType unknown')

In [13]:
def generate_test_database_postinsert():
    ### Generate database just as before
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    
    d.collection.insert_many_by_dataframe(records.head(10), 'valid_from_date') #initial insert

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:          
        d.collection.execute_operation(operation[0],operation[1],operation[2])

    d.collection.insert_many_by_dataframe(records.head(-10), 'valid_from_date')    
    
    return d    

In [14]:
def update_and_read_test(percent_of_update):
    ### Generate database just as before    
    d = generate_test_database_preinsert()   
    original_records = d.records.copy()

    updates = math.floor(100*percent_of_update)
    reads = 100-updates

    sequence = ([True]*updates)
    sequence.extend([False]*reads)
    random.shuffle(sequence)    

    records = [d.generate_record() for i in range(updates)]
    records_2 = records.copy()

    queries = []   

    for i in range(reads):        
        field = (random.choice(d.fields))[0]
        value = random.choice(d.field_domain[field])
        queries.append({field:value})   

    queries_2 = queries.copy()     

    start = time.time()
    for operation in sequence:             
        if operation: # insert - Nos nossos casos de uso não faz muito sentido deleções e updates. 
            record = records.pop()
            start_2 = time.time()
            d.collection.insert_one(json.dumps(record, default=str),record['valid_from_date'])                        
            end_2 = time.time()
            #print('Insertion time:' + str(end_2-start_2))
        else:
            start_2 = time.time()
            d.collection.find_many(queries.pop()) ##Nao to considerando a presença ou ausência de índices
            end_2 = time.time()
            print('Query time:' + str(end_2-start_2))

    end = time.time()
    print("Operations time: "+ str(end-start))
    ##Evolute everything in the end            
        
    end = time.time()
    preinsered_time = end-start
    d.destroy()

    client = MongoClient(host)        
    db = client[d.database_name]
    base_collection = db[d.collection_name]

    base_collection.insert_many(original_records)

    start = time.time()    
    for operation in sequence:             
        if operation: 
            record = records_2.pop()
            base_collection.insert_one(record)                      
        else:
            base_collection.find(queries_2.pop()) ##Isso nao faz exatamente sentido. Deveria gerar uma nova query 
    end = time.time()    
    baseline_time = (end-start)
    client.drop_database(d.database_name)
    
    return ({'preinserted': preinsered_time, 'baseline': baseline_time})
    

### Scenario 0 - Full Insertion - Lazy insertion

In [15]:
for i in range(number_of_tests):
    time_taken = update_and_read_test(1)
    performance_results = performance_results.append({'stage' : 'Full Insertion', 'experiment': 'baseline','time':time_taken['baseline']}, ignore_index=True)
    performance_results = performance_results.append({'stage' : 'Full Insertion', 'experiment': 'insert-first','time':time_taken['preinserted']}, ignore_index=True)

Operations time: 1.9214556217193604
Operations time: 1.7183477878570557
Operations time: 1.7345561981201172
Operations time: 1.890660285949707
Operations time: 1.6057016849517822


In [17]:
performance_results

,stage,experiment,time
0,LoadingPhase,insert-first,19.609344
1,LoadingPhase,insert-first,19.462202
2,LoadingPhase,insert-first,19.905794
3,LoadingPhase,insert-first,22.676517
4,LoadingPhase,insert-first,22.605921
5,LoadingPhase,operations-first,12.405283
6,LoadingPhase,operations-first,11.088462
7,LoadingPhase,operations-first,10.675926
8,LoadingPhase,operations-first,9.451641
9,LoadingPhase,operations-first,12.907995


In [18]:
for i in range(number_of_tests):
    time_taken = update_and_read_test(0.5)
    performance_results = performance_results.append({'stage' : '50/50', 'experiment': 'baseline','time':time_taken['baseline']}, ignore_index=True)
    performance_results = performance_results.append({'stage' : '50/50', 'experiment': 'insert-first','time':time_taken['preinserted']}, ignore_index=True)

Query time:0.0014934539794921875
Query time:0.0013651847839355469
Query time:0.0016543865203857422
Query time:0.001516103744506836
Query time:0.0012781620025634766
Query time:0.0014071464538574219
Query time:0.0012536048889160156
Query time:0.001495361328125
Query time:0.0014071464538574219
Query time:0.0012462139129638672
Query time:0.001333475112915039
Query time:0.0012824535369873047
Query time:0.0011913776397705078
Query time:0.0009872913360595703
Query time:0.0015451908111572266
Query time:0.0013477802276611328
Query time:0.0012798309326171875
Query time:0.002087116241455078
Query time:0.0015702247619628906
Query time:0.00140380859375
Query time:0.0011034011840820312
Query time:0.0015325546264648438
Query time:0.0011594295501708984
Query time:0.0013191699981689453
Query time:0.0023941993713378906
Query time:0.0014476776123046875
Query time:0.002719879150390625
Query time:0.0012705326080322266
Query time:0.0014977455139160156
Query time:0.0012199878692626953
Query time:0.0015232563

In [19]:
performance_results

,stage,experiment,time
0,LoadingPhase,insert-first,19.609344
1,LoadingPhase,insert-first,19.462202
2,LoadingPhase,insert-first,19.905794
3,LoadingPhase,insert-first,22.676517
4,LoadingPhase,insert-first,22.605921
5,LoadingPhase,operations-first,12.405283
6,LoadingPhase,operations-first,11.088462
7,LoadingPhase,operations-first,10.675926
8,LoadingPhase,operations-first,9.451641
9,LoadingPhase,operations-first,12.907995


### Scenario 2 - Read Heavy

In this scenario, 95% of operations executed are read operations, while 5% are write operations (insertion).

In [20]:
for i in range(number_of_tests):
    time_taken = update_and_read_test(0.05)
    performance_results = performance_results.append({'stage' : 'ReadHeavy', 'experiment': 'baseline','time':time_taken['baseline']}, ignore_index=True)
    performance_results = performance_results.append({'stage' : 'ReadHeavy', 'experiment': 'insert-first','time':time_taken['preinserted']}, ignore_index=True)

Query time:0.0019145011901855469
Query time:0.001176595687866211
Query time:0.001096963882446289
Query time:0.0012404918670654297
Query time:0.0012409687042236328
Query time:0.0013015270233154297
Query time:0.001363515853881836
Query time:0.001163482666015625
Query time:0.001203775405883789
Query time:0.0012547969818115234
Query time:0.0012485980987548828
Query time:0.0012059211730957031
Query time:0.0012230873107910156
Query time:0.0010089874267578125
Query time:0.0009515285491943359
Query time:0.0009469985961914062
Query time:0.0009436607360839844
Query time:0.0010166168212890625
Query time:0.0009851455688476562
Query time:0.000990152359008789
Query time:0.000993967056274414
Query time:0.0010199546813964844
Query time:0.0009784698486328125
Query time:0.0010297298431396484
Query time:0.0010683536529541016
Query time:0.0012173652648925781
Query time:0.0010881423950195312
Query time:0.0010831356048583984
Query time:0.0011854171752929688
Query time:0.0011792182922363281
Query time:0.0011

### Scenario 3 - WriteHeavy

In this scenario, 95% of operations executed are write operations, while 5% are read operations.

In [22]:
for i in range(number_of_tests):
    time_taken = update_and_read_test(0.95)
    performance_results = performance_results.append({'stage' : 'WriteHeavy', 'experiment': 'baseline','time':time_taken['baseline']}, ignore_index=True)
    performance_results = performance_results.append({'stage' : 'WriteHeavy', 'experiment': 'insert-first','time':time_taken['preinserted']}, ignore_index=True)

Query time:0.0014307498931884766
Query time:0.0016067028045654297
Query time:0.0013704299926757812
Query time:0.0015037059783935547
Query time:0.001346588134765625
Operations time: 1.907688856124878
Query time:0.0010061264038085938
Query time:0.0014052391052246094
Query time:0.00119781494140625
Query time:0.0008189678192138672
Query time:0.0010852813720703125
Operations time: 1.852182149887085
Query time:0.0015192031860351562
Query time:0.0009980201721191406
Query time:0.0015895366668701172
Query time:0.0012500286102294922
Query time:0.0015652179718017578
Operations time: 1.738072395324707
Query time:0.001474618911743164
Query time:0.0015308856964111328
Query time:0.0014200210571289062
Query time:0.0026178359985351562
Query time:0.0015060901641845703
Operations time: 1.8655192852020264
Query time:0.0017235279083251953
Query time:0.0011296272277832031
Query time:0.0011684894561767578
Query time:0.0015022754669189453
Query time:0.0014545917510986328
Operations time: 1.8172903060913086


In [23]:
performance_results

,stage,experiment,time
0,LoadingPhase,insert-first,19.609344
1,LoadingPhase,insert-first,19.462202
2,LoadingPhase,insert-first,19.905794
3,LoadingPhase,insert-first,22.676517
4,LoadingPhase,insert-first,22.605921
5,LoadingPhase,operations-first,12.405283
6,LoadingPhase,operations-first,11.088462
7,LoadingPhase,operations-first,10.675926
8,LoadingPhase,operations-first,9.451641
9,LoadingPhase,operations-first,12.907995
